# How to run genome alignment

Purpose is to map raw reads to reference genome.

Input:
 * fastq files (raw reads)

Output:
 * bam alignment file (aligned to reference genome)
 * bam tabix index file for fast chrom and position range lookups

## Dependencies

### Install

`bowtie2` and `samtools`

Example using bioconda:
 * https://anaconda.org/bioconda/samtools
 * https://anaconda.org/bioconda/bowtie2

### Download reference genome (hg38)

Download `GCA_000001405.15_GRCh38_full_analysis_set.fna.gz`, `GCA_000001405.15_GRCh38_full_analysis_set.fna.fai` and `GCA_000001405.15_GRCh38_full_analysis_set.fna.bowtie_index.tar.gz` (need to unzip it before using) files from [ncbi](ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.22_GRCh38.p7/GRCh38_major_release_seqs_for_alignment_pipelines).

## Run the alignment

If running locally make sure you have around 10-40h to spare and 300GB of free hard disk (depends on hardware).

For example it took my desktop i7-10700KF 16h to complete in 14 threads, for sorting HDD was the main limiting factor.

## Running outside of jupyter

jupyter has its own resource limitations and it might be a hassle to configure them to run commands below. I pulled commands below into a bash script for convenience - see [this script](./bin/raw_reads_to_alignment.sh) for details.

Example usage:
```
./bin/raw_reads_to_alignment.sh 6 /path/to/ref_gen/index_bowtie_GRCh38.p7.tar.gz /path/to/data/my_genome_fastq_001.fastq.gz /path/to/data/my_genome_fastq_002.fastq.gz /output
```

In [ ]:
%%time
! #bowtie2-build /path/to/ref_gen/GCA_000001405.22_GRCh38.p7_genomic.fna.gz /path/to/ref_gen/index_bowtie_GRCh38.p7 # create bowtie2 index if you didn't download it\n
! bowtie2 -p 7 -x /path/to/ref_gen/index_bowtie_GRCh38.p7 -1 /path/to/data/my_genome_fastq_001.fastq.gz -2 /path/to/data/my_genome_fastq_002.fastq.gz | samtools view -bS - > /path/to/data/my_genome_hg38.p7.unsorted.bam # p for number of process; -x index refers to the bowtie index folder

Need to sort alignment results and make them easily searchable (create tabix index which enables quick read lookups based on contig and position range).

In [ ]:
%%time
! samtools sort -@ 7 /path/to/data/GENOME12345_hg38.p7.unsorted.bam -o /path/to/data/GENOME12345_hg38.p7.bam # decreases the size of the alignment file to around 40GB; -@ for number of parallel threads\n",
! samtools index -@ 7 /path/to/data/GENOME12345_hg38.p7.bam # created GENOME12345_hg38.p7.bam.bai index file for fast lookups; -@ for number of parallel threads"